# Clustering 3k PBMCs following a Seurat Tutorial

This started out (July 2017) with a demonstration that Scanpy would allow to reproduce most of Seurat's ([Satija *et al.*, 2015](https://doi.org/10.1038/nbt.3192)) clustering tutorial ([link](http://satijalab.org/seurat/pbmc3k_tutorial.html)), which we gratefully acknowledge. In the meanwhile, we have added and removed several pieces.

The data consists in *3k PBMCs from a Healthy Donor* and is freely available from 10x Genomics ([here](http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz) from this [webpage](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k)).

In [ ]:
import numpy as np
import pandas as pd
import scanpy.api as sc
from scipy import sparse, io
from collections import Counter

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)

## Create input adata

In [ ]:
df = pd.read_csv("data/NeuronalGeneCount.csv", index_col=0)
/print df.shape

df = df.drop_duplicates(keep='first').T
/print df.shape
df.to_pickle('data/df.pkl')
df.head()

In [ ]:
df = pd.read_pickle('data/df.pkl')

In [ ]:
np.save('data/cells.npy', df.index.values)
np.save('data/genes.npy', df.columns)

In [ ]:
m = sparse.csr_matrix(df.as_matrix().T)

io.mmwrite("data/matrix.mtx", m)
del m, df

## Start from here if adata is available

In [ ]:
%%time
path = './data/'
adata = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
adata.var_names = np.load('data/genes.npy')
adata.obs_names = np.load('data/cells.npy')

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata.shape

## Preprocessing

**Note:** In notebooks and jupyter lab, you can see the documentation for a python function by hitting ``SHIFT + TAB``. Hit it twice to expand the view.

Show those genes that yield the highest fraction of counts in each single cells, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Basic filtering.

In [ ]:
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.shape

Plot some information about mitochondrial genes, important for quality control. Note that you can also retrieve mitochondrial genes using `sc.queries.mitochondrial_genes_biomart('www.ensembl.org', 'mmusculus')`.

Citing from "Simple Single Cell" workflows [(Lun, McCarthy & Marioni, 2017)](https://master.bioconductor.org/packages/release/workflows/html/simpleSingleCell.html#examining-gene-level-metrics):
> High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.

In [ ]:
mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse to transform to a dense array after summing
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

A violin plot of the computed quality measures.

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

Remove cells that have too many mitochondrial genes expressed or too many total counts.

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
adata.shape

Actually do the filtering.

In [ ]:
adata = adata[adata.obs['n_genes'] < 1250, :]
adata = adata[adata.obs['percent_mito'] < 0.05, :]

Set the `.raw` attribute of AnnData object to the logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object returned by `sc.pp.log1p`.

In [ ]:
adata.raw = sc.pp.log1p(adata, copy=True)

Per-cell normalize the data matrix $\mathbf{X}$. Many people would consider the normalized data matrix as the "relevant data" for visualization and differential testing (assessing feature importance). Until a common viewpoint is reached on this, the decision of what to consider "raw", is up to the user. We tend to recommend to use the normalized data for visualization and differential testing even though here, we use the non-normalized data for the sake of consistency with the Seurat tutorial.

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)

Identify highly-variable genes.

In [ ]:
filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.005, max_mean=6, min_disp=0.1)
sc.pl.filter_genes_dispersion(filter_result)

Actually do the filtering.

In [ ]:
adata = adata[:, filter_result.gene_subset]
adata.shape

Logarithmize the data.

In [ ]:
sc.pp.log1p(adata)

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10. 

In [ ]:
sc.pp.scale(adata, max_value=10)

Save the result.

## PCA

Compute PCA and make a scatter plot.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata)

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often, a rough estimate of the number of PCs does fine. Seurat provides many more functions, here.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

We now advertise visualizing the data using UMAP, see below. In particular, if you have large data, this will give you a notable speedup. Also, it is potentially more faithful to global topology: trajectories are better preserved.

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata)

As we set the `.raw` attribute of AnnData (a "frozen" state of the object at a point in the pipeline where we deemed the data "raw"), the previous plots showed the raw gene expression. You can also plot the scaled and corrected gene expression by explicitly stating that you don't want to use `.raw`.

In [ ]:
sc.pl.umap(adata, use_raw=False)

## Clustering the graph

As Seurat and many others, we recommend the Louvain graph-clustering method (community detection based on optimizing modularity). It has been proposed for single-cell data by [Levine *et al.* (2015)](https://doi.org/10.1016/j.cell.2015.05.047). Note that Louvain clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
sc.tl.louvain(adata)

Plot the data with tSNE. Coloring according to clustering. Clusters agree quite well with the result of Seurat.

In [ ]:
sc.pl.umap(adata, color=['louvain'], save='/bla.png')

In [ ]:
adata.obs['louvain'].value_counts()

In [ ]:
adata.obs['louvain'].to_csv("seurat.csv", header = False)

## Finding marker genes

In [ ]:
adata.obs['louvain'].shape

Let us compute a ranking for the highly differential genes in each cluster. Here, we simply rank genes with a t test, which agrees quite well with Seurat.

For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before.

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, save='/test.pdf')

# Group all calls into one function

In [ ]:
def run(df, prefix = 'all_no_impute', 
        min_genes=10, min_cells=3, min_mean=0.005, max_mean=6, min_disp=0.1, n_neighbors=10, n_pcs=40):
    np.save(f'data/{prefix}_cells.npy', df.index.values)
    np.save(f'data/{prefix}_genes.npy', df.columns)
    m = sparse.csr_matrix(df.values.T)
    io.mmwrite(f"data/{prefix}_matrix.mtx", m)
    del m
    adata = sc.read(f"data/{prefix}_matrix.mtx", cache=True).T  # transpose the data
    adata.var_names = np.load(f'data/{prefix}_genes.npy')
    adata.obs_names = np.load(f'data/{prefix}_cells.npy')
    adata.var_names_make_unique()
    print('Original size',adata.shape)
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    adata.raw = sc.pp.log1p(adata, copy=True)
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=min_mean, max_mean=max_mean, min_disp=min_disp)
    sc.pl.filter_genes_dispersion(filter_result)
    adata = adata[:, filter_result.gene_subset]
    print('After dispersion filter',adata.shape)
    sc.pp.log1p(adata) # log transform
    sc.pp.regress_out(adata, ['n_counts'])
    sc.pp.scale(adata, max_value=10) # clip values with std > max_value
    # PCA transform
    sc.tl.pca(adata, svd_solver='arpack')
    
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)
    sc.tl.umap(adata)
    
    sc.tl.louvain(adata)
    sc.pl.umap(adata, color=['louvain'], save=f'/{prefix}.png')
    print(adata.obs['louvain'].value_counts())
    with open("cluster_sizes.txt", "a") as myfile:
        myfile.write(f"\n\n{prefix} : \n{adata.obs['louvain'].value_counts()}")
    sc.tl.rank_genes_groups(adata, 'louvain')
    sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, save=f'/{prefix}.png')
    return adata

def getCellsFromCluster(adata, clusterId = '0'):
    v = adata.obs['louvain']
    index = np.where(v.values == clusterId)[0]
    return v.index[index]

In [ ]:
df = pd.read_pickle('data/df.pkl')

In [ ]:
adata = run(df, prefix = 'all_no_impute')

# Split clusters

In [ ]:
for clusterId in ['0', '1']:
    clusterCells = getCellsFromCluster(adata, clusterId = clusterId)
    print(f'Cluster {clusterId} has {len(clusterCells)} cells')
    adata_t = run(df[df.index.isin(clusterCells)], prefix = f'no_impute_cluster{clusterId}')

# Use imputed data

In [ ]:
df = pd.read_pickle('data/imputed_df.pkl')

In [ ]:
adata = run(df, prefix = 'all_imputed')

In [ ]:
for clusterId in ['0', '1', '2', '3']:
    clusterCells = getCellsFromCluster(adata, clusterId = clusterId)
    print(f'Cluster {clusterId} has {len(clusterCells)} cells')
    adata_t = run(df[df.index.isin(clusterCells)], prefix = f'imputed_cluster{clusterId}')